In [1]:
## This notebook contains a crewai multiagentic system code that on providing the synopsis and language as input 
## will give a full book as output.

In [13]:
from crewai import Task, Agent, Crew, LLM, Process

In [5]:
# parameters
desired_language = 'Spanish'
total_words = 3000
no_of_chapters = 4
synopsis = ''

In [14]:
# ----------------------
# AGENTS
# ----------------------

# 1. Planner Agent
planner_agent = Agent(
    role="Senior Content Planner",
    goal = f"Develop a cohesive and well-structured book with {no_of_chapters} chapters based on the provided synopsis. Also give number of words to each chapter of around 2000 words.",
    backstory="You're tasked with planning the structure and content for a book. "
            f"The input provided is a synopsis that outlines the central idea or story: {synopsis}. "
            "Your role is to expand on the synopsis and create a detailed plan, including sections, chapters, "
            "or thematic elements, depending on the type of content. For books and novels, focus on structuring chapters, "
            "developing key events, and maintaining a coherent narrative arc. For poetry, focus on defining themes, tones, "
            "and stylistic elements like rhyme schemes or free verse. "
            "Your plan will serve as the foundation for the Content Writer to craft the final book.",
    memory=True,  # This enables memory for iterative tasks
    verbose=True
)

# 2. Writing Agent
writing_agent = Agent(
    role="Senior Content Writer.",
    goal= f"Write content for book sections in {desired_language}, ensuring relevance and clarity.",
    backstory="You excel at creating long-form content that is engaging, structured, and well-written.",
    memory=True,
    verbose=True
)

# 3. Proofreader Agent
proofreader_agent = Agent(
    role="Proofreader Agent",
    goal="Review content for grammar, spelling, and language consistency.",
    backstory="You are an expert in language mechanics, ensuring all content is polished and error-free.",
    memory=False,
    verbose=True
)

# 4. Editor Agent
editor_agent = Agent(
    role="Editor Agent",
    goal="Edit and refine chapters for coherence, accuracy, and narrative flow.",
    backstory="A perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
    memory=True,
    verbose=True
)

# 5. Memory Agent
memory_agent = Agent(
    role="Memory Agent",
    goal="Summarize completed chapters to provide context for subsequent chapters.",
    backstory="You maintain a concise summary of the book’s progress to ensure continuity across chapters.",
    memory=True,
    verbose=True
)

In [18]:
# ----------------------
# TASKS
# ----------------------

# 1. Generate Outline Task
generate_outline_task = Task(
    description="Create a detailed book outline with chapters and sections from the synopsis.",
    expected_output="A detailed outline with chapter titles and their respective sections.",
    agent=planner_agent
)

# 2. Write Section Task
write_section_task = Task(
    description=f"Write content for a specific section in {desired_language} as per the outline.",
    expected_output="A well-written section of the chapter.",
    agent=writing_agent,
    async_execution=True
)

# 3. Proofread Section Task
proofread_section_task = Task(
    description="Proofread the section to correct grammar, spelling, and language issues.",
    expected_output="A polished version of the section with no grammatical errors.",
    agent=proofreader_agent
)

# 4. Edit Chapter Task
edit_chapter_task = Task(
    description="Combine and refine sections into a cohesive and polished chapter.",
    expected_output="A completed chapter with seamless transitions and narrative flow.",
    agent=editor_agent
)

# 5. Summarize Chapter Task
summarize_chapter_task = Task(
    description="Summarize the completed chapter to provide context for the next chapter.",
    expected_output="A concise summary of the chapter’s content.",
    agent=memory_agent
)

In [17]:
# Input placeholder for the synopsis and initial parameters
inputs = {
    "synopsis": '''In the heart of a vibrant Spanish town, a young dreamer discovers a hidden journal that unravels the secrets of an ancient love story. 
                    As past and present collide, she embarks on a journey of self-discovery, uncovering truths about her heritage and the power of destiny. 
                    Rich with cultural charm and emotion, this tale celebrates the resilience of love and the beauty of embracing one's roots.''',
    "desired_language": desired_language,
    "total_words" : 3000,
    "no_of_chapters" : 4 # Define the total number of chapters for the book
}

# Initialize the Crew with agents and tasks
crew = Crew(
    agents=[planner_agent, writing_agent, proofreader_agent, editor_agent, memory_agent],
    tasks=[
        generate_outline_task,
        write_section_task,
        proofread_section_task,
        edit_chapter_task,
        summarize_chapter_task
    ]
    #process=Process.custom  # Allows a custom orchestration of tasks
)

# ----------------------
# PROCESS LOGIC
# ----------------------

def execute_book_writing(inputs):
    """
    Executes the book-writing process iteratively for each chapter.
    """

    # Step 1: Generate the book outline
    outline_result = generate_outline_task.run(inputs={"synopsis": inputs["synopsis"]})
    book_outline = outline_result["output"]
    print(f"Generated Outline: {book_outline}")

    completed_book = ""
    chapter_summaries = []

    # Step 2: Process each chapter iteratively
    for chapter_number in range(1, inputs["total_chapters"] + 1):
        chapter_outline = book_outline[f"Chapter {chapter_number}"]

        # Write sections of the chapter
        chapter_content = ""
        for section in chapter_outline["sections"]:
            section_result = write_section_task.run(inputs={"section_outline": section})
            section_content = section_result["output"]

            # Proofread the section
            proofread_result = proofread_section_task.run(inputs={"section_content": section_content})
            polished_section = proofread_result["output"]

            # Append section to chapter content
            chapter_content += f"{polished_section}\n"

        # Edit the chapter
        edited_chapter_result = edit_chapter_task.run(inputs={"chapter_content": chapter_content})
        edited_chapter = edited_chapter_result["output"]

        # Append chapter to the completed book
        completed_book += f"{edited_chapter}\n\n"

        # Summarize the chapter
        summary_result = summarize_chapter_task.run(inputs={"chapter_content": edited_chapter})
        chapter_summary = summary_result["output"]

        # Store the summary for the next chapter
        chapter_summaries.append(chapter_summary)

        # Update inputs for the next chapter with the summary
        inputs["context"] = " ".join(chapter_summaries)

        print(f"Completed Chapter {chapter_number}")

    print("Book Writing Process Completed!")
    return completed_book


# Run the process
final_book = execute_book_writing(inputs=inputs)

# Save the final book to a file
with open("final_book.txt", "w", encoding="utf-8") as book_file:
    book_file.write(final_book)

print("The book has been written and saved to 'final_book.txt'.")

AttributeError: 'Task' object has no attribute 'run'

ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/karthikreddy/crewai_agents/crewai_venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/karthikreddy/crewai_agents/crewai_venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/karthikreddy/crewai_agents/crewai_venv/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/karthikreddy/.pyenv/versions/3.12.0/lib/python3.12/http/client.py", line 1411, in getresponse
    response.begin()
  File "/Users/karthikreddy/.pyenv/versions/3.12.0/lib/python3.12/http/client.py", line 324, in begin
    version, status, reason 

In [78]:
#### Planning agent
planner_agent = Agent(
    role="Senior Content Planner",
    goal = f"Develop a cohesive and well-structured outline for a book in spanish language with {no_of_chapters} chapters based on the provided synopsis : {synopsis}"
            "Each chapter should include a detailed breakdown into sections.",
    backstory="You're tasked with planning the structure and content for a book. "
            "The input provided is the synopsis that outlines the central idea or story."
            "Your role is to expand on the synopsis and create a detailed plan, including sections, chapters, "
            "or thematic elements, depending on the type of content. For books and novels, focus on structuring chapters, "
            "developing key events, and maintaining a coherent narrative arc. For poetry, focus on defining themes, tones, "
            "and stylistic elements like rhyme schemes or free verse. "
            "Your plan will serve as the foundation for the Content Writer to craft the final book.",
    memory=True,  # This enables memory for iterative tasks
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

generate_outline_task = Task(
    description="Create a detailed book outline with chapters and sections from the synopsis in spanish language.",
    expected_output="A detailed outline in spanish language with chapter titles and their respective sections.",
    agent=planner_agent
)

crew = Crew(agents=[planner_agent],
           tasks=[generate_outline_task])

In [79]:
synopsis = '''In the heart of a vibrant Spanish town, a young dreamer discovers a hidden journal that unravels the secrets of an ancient love story. 
            As past and present collide, she embarks on a journey of self-discovery, uncovering truths about her heritage and the power of destiny. 
            Rich with cultural charm and emotion, this tale celebrates the resilience of love and the beauty of embracing one's roots.''',


inputs = {"no_of_chapters":4,
         "synopsis":synopsis}

In [80]:
results = crew.kickoff(inputs)

# Agent: Senior Content Planner
## Task: Create a detailed book outline with chapters and sections from the synopsis in spanish language.


# Agent: Senior Content Planner
## Final Answer: 
Capítulo 1: El Descubrimiento
   Sección 1: Encuentro con el diario antiguo
   Sección 2: La primera revelación
   Sección 3: El inicio de la búsqueda
   
Capítulo 2: El Viaje al Pasado
   Sección 1: Misterios del pueblo
   Sección 2: Encuentros inesperados
   Sección 3: Revelaciones sorprendentes
   
Capítulo 3: El Poder del Destino
   Sección 1: La conexión con el pasado
   Sección 2: Decisiones cruciales
   Sección 3: El camino hacia la verdad
   
Capítulo 4: El Descubrimiento de Sí Misma
   Sección 1: Revelaciones sobre su herencia
   Sección 2: Aceptación y crecimiento personal
   Sección 3: Celebración de sus raíces y del amor  

This outline establishes a structured progression for the book, guiding the reader through the journey of the young dreamer as she unravels the secrets of the ancient

In [81]:
outline = results.raw
desired_language = 'Spanish'

In [82]:
### Writer , Proof reader, editor and summariser.

writing_agent = Agent(
    role="Senior Content Writer.",
    goal= f"Write content for only chapter 1 from the given outline : {outline} and synopsis : {synopsis} in the {desired_language} language for around 2000 words, ensuring relevance and clarity."
            "and maintaining the same structure as given in the outline"
            "Include section names in the content as per the outline.",
    backstory="You excel at creating long-form content that is engaging, structured, and well-written.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)


proofreader_agent = Agent(
    role="Senior Content Proofreader",
    goal="Review the chapter content for grammar, spelling, language consistency, coherence and style",
    backstory="You are an expert in language mechanics, ensuring all content is polished and error-free.",
    memory=False,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

editor_agent = Agent(
    role="Senior Content Editor",
    goal="Edit and refine chapters for coherence, accuracy, and narrative flow and anyother feedback from Senior Content Proofreader",
    backstory="A perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

write_section_task = Task(
    description="Write content for about 2000 words for the chapters with the given outline and synopsis.",
    expected_output="A well-written chapter of the book.",
    agent=writing_agent,
    async_execution=True
)

# 3. Proofread Section Task
proofread_section_task = Task(
    description="Proofread the section to correct grammar, spelling, language, coherence and style issues.",
    expected_output="A polished version of the section with no grammatical errors.",
    agent=proofreader_agent
)

# 4. Edit Chapter Task
edit_chapter_task = Task(
    description="Combine and refine sections into a cohesive and polished chapter.",
    expected_output="A completed chapter with sections, seamless transitions and narrative flow.",
    agent=editor_agent
)

In [83]:
chapter1_crew = Crew(agents=[writing_agent,proofreader_agent,editor_agent],
                    tasks=[write_section_task,proofread_section_task,edit_chapter_task])

In [84]:
# chpater_1_inputs = {"outline":outline}

chapter_1_content = chapter1_crew.kickoff().raw

# Agent: Senior Content Writer.
## Task: Write content for about 2000 words for the chapters with the given outline and synopsis.


# Agent: Senior Content Writer.
## Final Answer: 
Capítulo 1: El Descubrimiento

Sección 1: Encuentro con el diario antiguo

En el corazón de un vibrante pueblo español, donde las calles resonaban con la historia susurrada por el viento, una joven soñadora llamada Sofia tropezó con un hallazgo inesperado que cambiaría su vida para siempre. Mientras exploraba un mercado local, sus ojos se posaron sobre un antiguo diario encuadernado en cuero desgastado. El tacto áspero de las páginas amarillentas despertó su curiosidad, y sin dudarlo, decidió adquirir aquel tesoro del pasado.

Al abrir el diario, se encontró con una caligrafía elegante que narraba la historia prohibida de un amor perdido en el tiempo. Las palabras impregnadas de emoción y secretos susurraban al alma de Sofia, como si el destino hubiera conspirado para guiarla hacia ese momento preciso. Cada

In [85]:
memory_agent = Agent(role = "Content Memory Keeper",
                    goal  = f"Summarise the given chapter : {chapter_1_content} so as to maintain the story continuity , key themes, character plot.",
                    backstory="You are responsible for keeping track of the story, plot and key themes and character plots.",
                    verbose=True,
                    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key'))

memory_task = Task(description="Summarise the given chapter without losing the context, story theme, coherence",
                expected_output="A summarised version of the given chapter, so it will be passed to next chapter to maintian coherence, relevance etc.,",
                agent=memory_agent)

memory_crew = Crew(agents=[memory_agent],
                  tasks=[memory_task])

In [86]:
summarised_chapter_1 = memory_crew.kickoff().raw

# Agent: Content Memory Keeper
## Task: Summarise the given chapter without losing the context, story theme, coherence


# Agent: Content Memory Keeper
## Final Answer: 
En el corazón de un vibrante pueblo español, donde las calles resonaban con la historia susurrada por el viento, una joven soñadora llamada Sofía tropezó con un hallazgo inesperado que cambiaría su vida para siempre. Mientras exploraba un mercado local, sus ojos se posaron sobre un antiguo diario encuadernado en cuero desgastado. El tacto áspero de las páginas amarillentas despertó su curiosidad y, sin dudarlo, decidió adquirir aquel tesoro del pasado.

Al abrir el diario, se encontró con una caligrafía elegante que narraba la historia prohibida de un amor perdido en el tiempo. Las palabras impregnadas de emoción y secretos susurraban al alma de Sofía, como si el destino hubiera conspirado para guiarla hacia ese momento preciso. Cada página era un portal a un mundo olvidado, donde los suspiros de amantes separados se m

"In the heart of a small town, young Aryan's dreams of cricket stardom took root amidst humble beginnings. Despite challenges with lack of resources, mentors like Mr. Kapoor guided him, fueling his passion. Aryan's tale of resilience and determination is just beginning, promising a cricketing journey unlike any other."

In [87]:
### Writer , Proof reader, editor and summariser.

writing_agent_2 = Agent(
    role="Senior Content Writer.",
    goal= f"Write content for chapter 2, given chapter 1 summary : {summarised_chapter_1} "
            f"and outline : {outline} and synopsis : {synopsis} in the {desired_language} language for around 2000 words, ensuring relevance and clarity."
            "and maintaining the same structure as given in the outline"
            "Include section names in the content as per the outline.",
    backstory="You excel at creating long-form content that is engaging, structured, and well-written.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)


proofreader_agent_2 = Agent(
    role="Senior Content Proofreader",
    goal="Review the chapter content for grammar, spelling, language consistency, coherence and style",
    backstory="You are an expert in language mechanics, ensuring all content is polished and error-free.",
    memory=False,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

editor_agent_2 = Agent(
    role="Senior Content Editor",
    goal="Edit and refine chapters for coherence, accuracy, and narrative flow and anyother feedback from Senior Content Proofreader",
    backstory="A perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

write_section_task_2 = Task(
    description="Write content for the chapter 2 with the given outline, synopsis and chapter 1 summary",
    expected_output="A well-written chapter of the book.",
    agent=writing_agent_2,
    async_execution=True
)

# 3. Proofread Section Task
proofread_section_task_2 = Task(
    description="Proofread the section to correct grammar, spelling, language, coherence and style issues.",
    expected_output="A polished version of the section with no grammatical errors.",
    agent=proofreader_agent_2
)

# 4. Edit Chapter Task
edit_chapter_task_2 = Task(
    description="Combine and refine sections into a cohesive and polished chapter.",
    expected_output="A completed chapter with sections, seamless transitions and narrative flow.",
    agent=editor_agent_2
)

chapter2_crew = Crew(agents=[writing_agent_2,proofreader_agent_2,editor_agent_2],
                    tasks=[write_section_task_2,proofread_section_task_2,edit_chapter_task_2])

chapter_2_content = chapter2_crew.kickoff().raw

# Agent: Senior Content Writer.
## Task: Write content for the chapter 2 with the given outline, synopsis and chapter 1 summary


# Agent: Senior Content Writer.
## Final Answer: 
**Capítulo 2: El Viaje al Pasado**

En el corazón de un vibrante pueblo español, donde el tiempo parecía detenerse entre las sombras de las antiguas murallas, Sofía se preparaba para adentrarse en un viaje que trascendería las fronteras del presente. Con el diario antiguo entre sus manos temblorosas, la joven soñadora se dispuso a explorar los misterios ocultos que aguardaban más allá de las calles empedradas que una vez llamó hogar.

**Sección 1: Misterios del Pueblo**

El sol dorado se alzaba sobre los tejados rojos, iluminando el camino de Sofía mientras se adentraba en los recovecos olvidados de su pueblo natal. Cada esquina parecía susurrar antiguos secretos, cada sombra danzaba con la promesa de revelaciones enterradas en el tiempo. Siguiendo las pistas dejadas por la caligrafía del diario, Sofía se vio

In [88]:
memory_agent_2 = Agent(role = "Content Memory Keeper",
                    goal  = f"Summarise the given chapter : {chapter_2_content} so as to maintain the story continuity , key themes, character plot.",
                    backstory="You are responsible for keeping track of the story, plot and key themes and character plots.",
                    verbose=True,
                    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key'))

memory_task_2 = Task(description="Summarise the chapter without losing the context, story theme, coherence",
                expected_output="A summarised version of the given chapter, so it will be passed to next chapter to maintian coherence, relevance etc.,",
                agent=memory_agent_2)

memory_crew_2 = Crew(agents=[memory_agent_2],
                  tasks=[memory_task_2])

summarised_chapter_2 = memory_crew_2.kickoff().raw

# Agent: Content Memory Keeper
## Task: Summarise the chapter without losing the context, story theme, coherence


# Agent: Content Memory Keeper
## Final Answer: 
En el corazón de un vibrante pueblo español, Sofía se preparaba para adentrarse en un viaje hacia el pasado a través de un diario antiguo. Mientras seguía las pistas del diario, descubría secretos enterrados en las calles empedradas de su hogar. En su odisea, Sofía se encontraba con personajes cuyas vidas se conectaban con la historia ancestral del lugar. Finalmente, en una cripta olvidada, Sofía hizo una revelación impactante que la llevó a comprender que su viaje apenas comenzaba y que cada secreto revelado era solo el comienzo de un descubrimiento que transformaría su vida y la historia del pueblo. Con el diario como guía, se preparaba para seguir explorando lo desconocido, marcada por el poder del amor y la herencia que la había llevado hasta allí.




In [93]:
desired_language

'Spanish'

In [94]:
### Writer , Proof reader, editor and summariser.

writing_agent_3 = Agent(
    role="Senior Content Writer.",
    goal= f"Write content for chapter 3, given chapter 1 summary : {summarised_chapter_1}, chapter 2 summary : {summarised_chapter_2} "
            f"and outline : {outline} and synopsis : {synopsis} in the {desired_language} language for around 2000 words, ensuring relevance and clarity."
            "and maintaining the same structure as given in the outline"
            "Include section names in the content as per the outline.",
    backstory="You excel at creating long-form content that is engaging, structured, and well-written.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)


proofreader_agent_3 = Agent(
    role="Senior Content Proofreader",
    goal="Review the chapter content for grammar, spelling, language consistency, coherence and style",
    backstory="You are an expert in language mechanics, ensuring all content is polished and error-free.",
    memory=False,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

editor_agent_3 = Agent(
    role="Senior Content Editor",
    goal="Edit and refine chapters for coherence, accuracy, and narrative flow and anyother feedback from Senior Content Proofreader",
    backstory="A perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

write_section_task_3 = Task(
    description=f"Write content in {desired_language}for the chapter 3 of around 2000 words with the given outline, synopsis and chapter 1 summary and chapter 2 summary",
    expected_output="A well-written chapter of the book.",
    agent=writing_agent_3,
    async_execution=True
)

# 3. Proofread Section Task
proofread_section_task_3 = Task(
    description="Proofread the section to correct grammar, spelling, language, coherence and style issues.",
    expected_output="A polished version of the section with no grammatical errors.",
    agent=proofreader_agent_3
)

# 4. Edit Chapter Task
edit_chapter_task_3 = Task(
    description="Combine and refine sections into a cohesive and polished chapter.",
    expected_output="A completed chapter with sections, seamless transitions and narrative flow.",
    agent=editor_agent_3
)

chapter3_crew = Crew(agents=[writing_agent_3,proofreader_agent_3,editor_agent_3],
                    tasks=[write_section_task_3,proofread_section_task_3,edit_chapter_task_3])

chapter_3_content = chapter3_crew.kickoff().raw

memory_agent_3 = Agent(role = "Content Memory Keeper",
                    goal  = f"Summarise the given chapter : {chapter_3_content} so as to maintain the story continuity , key themes, character plot.",
                    backstory="You are responsible for keeping track of the story, plot and key themes and character plots.",
                    verbose=True,
                    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key'))

memory_task_3 = Task(description="Summarise the chapter without losing the context, story theme, coherence",
                expected_output="A summarised version of the given chapter, so it will be passed to next chapter to maintian coherence, relevance etc.,",
                agent=memory_agent_3)

memory_crew_3 = Crew(agents=[memory_agent_3],
                  tasks=[memory_task_3])

summarised_chapter_3 = memory_crew_3.kickoff().raw

# Agent: Senior Content Writer.
## Task: Write content in Spanishfor the chapter 3 of around 2000 words with the given outline, synopsis and chapter 1 summary and chapter 2 summary


# Agent: Senior Content Writer.
## Final Answer: 
Capítulo 3: El Poder del Destino

Sección 1: La conexión con el pasado

El sol se alzaba en el horizonte, iluminando las calles empedradas de aquel pueblo español imbuido en historia y misterio. Sofía llevaba consigo el diario antiguo, cuyas páginas relucían con la promesa de revelaciones futuras. Cada paso que daba resonaba con la certeza de que su destino estaba entrelazado con el pasado que yacía entre las sombras del olvido.

Al llegar a la plaza central, un antiguo reloj de sol marcaba el tiempo con la misma cadencia que los latidos de su corazón inquieto. Sofía se detuvo frente a una fuente centenaria, cuyas aguas susurraban antiguas melodías de amores perdidos y promesas no cumplidas. En ese instante, una conexión ancestral le envolvió, haciéndole se



# Agent: Senior Content Editor
## Final Answer: 
Capítulo 3: El Poder del Destino

El sol despertaba sobre el horizonte, iluminando las calles empedradas de un pueblo español cargado de historia y secretos. Sofía sostenía en sus manos el diario antiguo, cuyas páginas brillaban con promesas de revelaciones por venir. Cada paso que daba resonaba con la certeza de que su destino se entretejía con un pasado perdido en las sombras del olvido.

Al llegar a la plaza central, un antiguo reloj de sol marcaba el tiempo al ritmo de su inquieto corazón. Frente a una fuente centenaria, cuyas aguas murmuraban melodías de amores desvanecidos y promesas incumplidas, Sofía sintió una conexión ancestral envolverla. Como si las almas del pasado le susurraran al oído, se sintió guiada hacia la verdad que anhelaba descubrir.

El viento llevaba consigo susurros de un tiempo lejano, acariciando su rostro con ternura. En ese silencio etéreo, Sofía comprendió que el poder del destino habitaba en sus manos, e

In [95]:
### Writer , Proof reader, editor and summariser.

writing_agent_4 = Agent(
    role="Senior Content Writer.",
    goal= f"Write content for chapter 4, given chapter 1 summary : {summarised_chapter_1}, chapter 2 summary : {summarised_chapter_2} and chapter 3 summary : {summarised_chapter_3} "
            f"and outline : {outline} and synopsis : {synopsis} in the {desired_language} language for around 2000 words, ensuring relevance and clarity."
            "and maintaining the same structure as given in the outline"
            "Include section names in the content as per the outline.",
    backstory="You excel at creating long-form content that is engaging, structured, and well-written.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)


proofreader_agent_4 = Agent(
    role="Senior Content Proofreader",
    goal="Review the chapter content for grammar, spelling, language consistency, coherence and style",
    backstory="You are an expert in language mechanics, ensuring all content is polished and error-free.",
    memory=False,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

editor_agent_4 = Agent(
    role="Senior Content Editor",
    goal="Edit and refine chapters for coherence, accuracy, and narrative flow and anyother feedback from Senior Content Proofreader",
    backstory="A perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
    memory=True,
    verbose=True,
    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

write_section_task_4 = Task(
    description=f"Write content in {desired_language} language of about 2000 words for the chapter 4 with the given outline, synopsis and chapter 1 summary and chapter 2 summary",
    expected_output="A well-written chapter of the book.",
    agent=writing_agent_4,
    async_execution=True
)

# 3. Proofread Section Task
proofread_section_task_4 = Task(
    description="Proofread the section to correct grammar, spelling, language, coherence and style issues.",
    expected_output="A polished version of the section with no grammatical errors.",
    agent=proofreader_agent_4
)

# 4. Edit Chapter Task
edit_chapter_task_4 = Task(
    description="Combine and refine sections into a cohesive and polished chapter.",
    expected_output="A completed chapter with sections, seamless transitions and narrative flow.",
    agent=editor_agent_4
)

chapter4_crew = Crew(agents=[writing_agent_4,proofreader_agent_4,editor_agent_4],
                    tasks=[write_section_task_4,proofread_section_task_4,edit_chapter_task_4])

chapter_4_content = chapter4_crew.kickoff().raw

memory_agent_4 = Agent(role = "Content Memory Keeper",
                    goal  = f"Summarise the given chapter : {chapter_4_content} so as to maintain the story continuity , key themes, character plot.",
                    backstory="You are responsible for keeping track of the story, plot and key themes and character plots.",
                    verbose=True,
                    llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key'))

memory_task_4 = Task(description="Summarise the chapter without losing the context, story theme, coherence",
                expected_output="A summarised version of the given chapter, so it will be passed to next chapter to maintian coherence, relevance etc.,",
                agent=memory_agent_4)

memory_crew_4 = Crew(agents=[memory_agent_4],
                  tasks=[memory_task_4])

summarised_chapter_4 = memory_crew_4.kickoff().raw

# Agent: Senior Content Writer.
## Task: Write content in Spanish language of about 2000 words for the chapter 4 with the given outline, synopsis and chapter 1 summary and chapter 2 summary


# Agent: Senior Content Writer.
## Final Answer: 
En el corazón de un vibrante pueblo español, Sofía se enfrentaba a un momento crucial en su búsqueda de respuestas y redención. El diario antiguo en sus manos brillaba con una promesa de autodescubrimiento y revelaciones sobre su herencia que estaban por desplegarse ante ella.

### Capítulo 4: El Descubrimiento de Sí Misma

#### Sección 1: Revelaciones sobre su herencia
Después de adentrarse en el territorio desconocido del bosque de alcornoques centenarios, Sofía sintió que cada paso la llevaba más cerca de su destino final. Mientras seguía los susurros del diario antiguo y las indicaciones del anciano sabio, la joven soñadora se vio envuelta en una atmósfera cargada de misterio y emoción. Con el alma abierta a las revelaciones por venir, se encon



# Agent: Senior Content Editor
## Final Answer: 
En el corazón de un vibrante pueblo español, Sofía se enfrentaba a un momento crucial en su búsqueda de respuestas y redención. El diario antiguo en sus manos brillaba con una promesa de autodescubrimiento y revelaciones sobre su herencia que estaban por desplegarse ante ella.

### Capítulo 4: El Descubrimiento de Sí Misma

#### Sección 1: Revelaciones sobre su herencia
Después de adentrarse en el territorio desconocido del bosque de alcornoques centenarios, Sofía sintió que cada paso la llevaba más cerca de su destino final. Mientras seguía los susurros del diario antiguo y las indicaciones del anciano sabio, la joven soñadora se vio envuelta en una atmósfera cargada de misterio y emoción. Con el alma abierta a las revelaciones por venir, se encontró frente a un antiguo monasterio en ruinas, cuyas paredes parecían susurrar secretos ancestrales.

Al adentrarse en el monasterio, Sofía sintió el peso de la historia reposar sobre sus homb

In [101]:
### final book editor

book_editor_agent = Agent(
            role="Senior Content Editor",
            goal=f"Given different chapters, chapter 1: {chapter_1_content}, chapter 2: {chapter_2_content}, chapter 3: {chapter_3_content}, chapter 4: {chapter_4_content} " 
                f" Edit and refine these chapters in {desired_language} for a total of about 8000 to 10000 words for coherence, accuracy, and narrative flow to look similar in terms of structure and in other factors",
            backstory="A Spanish perfectionist with a knack for storytelling, you ensure all chapters are cohesive and engaging.",
            memory=True,
            verbose=True,
            llm=LLM(model='gpt-3.5-turbo', api_key='your-api-key')
)

book_editor_task = Task(
    description=f"Combine and refine all the chapters in {desired_language} for a total of about 8000 to 10000 words into a cohesive and polished chapter to maintain coninuity, well structure.",
    expected_output=f"A completed Book in {desired_language} with chapter and sections with seamless transitions and narrative flow."
                    "A marked down version so as to copy paste it to form a pdf book.",
    agent=book_editor_agent,
    output_file = 'ai_book.txt'
)

book_edit_crew = Crew(agents=[book_editor_agent],
                     tasks = [book_editor_task])

In [102]:
final_book = book_edit_crew.kickoff().raw

# Agent: Senior Content Editor
## Task: Combine and refine all the chapters in Spanish for a total of about 8000 to 10000 words into a cohesive and polished chapter to maintain coninuity, well structure.


# Agent: Senior Content Editor
## Final Answer: 
**Capítulo 1: El Descubrimiento**

En el corazón de un vibrante pueblo español, donde las calles resonaban con la historia susurrada por el viento, una joven soñadora llamada Sofía tropezó con un hallazgo inesperado que cambiaría su vida para siempre. Mientras exploraba un mercado local, sus ojos se posaron sobre un antiguo diario encuadernado en cuero desgastado. El tacto áspero de las páginas amarillentas despertó su curiosidad y, sin dudarlo, decidió adquirir aquel tesoro del pasado.

Al abrir el diario, se encontró con una caligrafía elegante que narraba la historia prohibida de un amor perdido en el tiempo. Las palabras impregnadas de emoción y secretos susurraban al alma de Sofía, como si el destino hubiera conspirado para guiarl

In [100]:
print(final_book)

**Capítulo Completo: El Descubrimiento de la Verdad**

En el corazón de un vibrante pueblo español, donde las calles resonaban con la historia susurrada por el viento, Sofía, una joven soñadora, tropezó con un hallazgo inesperado que cambiaría su vida para siempre. Mientras exploraba un mercado local, sus ojos se posaron sobre un antiguo diario encuadernado en cuero desgastado. El tacto áspero de las páginas amarillentas despertó su curiosidad y, sin dudarlo, decidió adquirir aquel tesoro del pasado.

Al abrir el diario, se encontró con una caligrafía elegante que narraba la historia prohibida de un amor perdido en el tiempo. Las palabras impregnadas de emoción y secretos susurraban al alma de Sofía, como si el destino hubiera conspirado para guiarla hacia ese momento preciso. Cada página era un portal a un mundo olvidado, donde los suspiros de amantes separados se mezclaban con la esencia del presente.

Con cada palabra leída, Sofía sentía cómo el velo del desconocimiento se desgarrab